# Sub-Sample Notebook

### Notes: 
- Datasets available
    - Accepted (1.68gig ~ 2.2M observations)
    - Rejected (1.78gig ~ 20M observations)
    
    
**- Must sub-sample datasets for Exploritory Data Analyis and Machine Learning**

### Load Packages

In [1]:
# Load packages
import pandas as pd
import numpy as np
import re
import math
from time import strptime  # format data columns
import random  # used in subsampling
import warnings

warnings.filterwarnings("ignore")  # ignore warnings throughout notebook

pd.set_option("display.max_columns", None)  # show all columns


# ======================================================
# WARNING: Following code used to auto-format notebook
# Must install nb_black package to run
# ======================================================
# pip install nb_black
%load_ext nb_black

### Helper Functions

In [51]:
def subsampling(
    input_path, output_path, csv_type, frac, chunksize, n_floor=False, printDetails=True
):

    """
    Purpose: 
        - Reduce dataframe size by randomly sub-sampling x% of observations (per year)
    Input:
        - input_path = path to csv to be sub-sampled
        - output_path = path to store processed csv
        - csv_type = 1 (accepted.csv) or 0 (declined.csv)
        - frac = fraction of observations to be sub-sampled each year
        - n_floor = set min numb. of subsample observations [n=20, per month]
        - printDetails = prints the following variables
            year processed
            size of subsampled observations for that year
            size of cumulative subsampled observations
    Return:
        - csv file saved to output_path
    """

    for yr in range(2007, 2019):

        # Import CSV into interable object of chunks
        # iter_csv = iteratore objectl; must recreate for every year
        iter_csv = pd.read_csv(
            input_path, header=0, iterator=True, low_memory=False, chunksize=chunksize,
        )
        
        #Define date feature to parse for subsampling
        if csv_type:
            date_feature = 'issue_d'
            year_idx = 1
        else:
            date_feature = "Application Date"
            year_idx = 0
    
        # Process CSV chunks per year
        year_store = pd.DataFrame()  # overwrite previous year-storage DataFrame
        for chunk in iter_csv:

            # Exclude loans missing issue date
            year = chunk[~chunk[date_feature].isna()]

            # Parse date feature into 'issue_year' column
            year["issue_year"] = year[date_feature].apply(
                    lambda x: int(re.split("-", x)[year_idx]))

            # Store data per year
            year_store = year_store.append(year[year["issue_year"] == yr])

        # Parse date feature into 'issue_month' column
        # csv_type=1 (Acceptance.csv), "issue_d" feature column
        # csv_type=0 (Decline.csv), "Application Date" feature column
        if csv_type:
            year_store["issue_month"] = year_store["issue_d"].apply(
                lambda x: strptime(re.split("-", x)[0], "%b").tm_mon
            )
        else:
            year_store["issue_month"] = year_store["Application Date"].apply(
                lambda x: int(re.split("-", x)[1])
            )

        # Generate list of randomly subsampled indices for each month in single year
        nest_subsamp_idx = [
            month_subsample_idx(year_store, frac, mo, n_floor) for mo in range(1, 13)
        ]  # returns nested list
        subsamp_idx = [
            item for elem in nest_subsamp_idx for item in elem
        ]  # flattens nested list

        # Store subsampled data per year
        if yr == 2007:
            df_stored = year_store.iloc[subsamp_idx]
        else:
            df_stored = df_stored.append(year_store.iloc[subsamp_idx])

        if printDetails:
            print("year = {0}".format(yr))
            print(
                "year_store length = {0}".format(year_store.iloc[subsamp_idx].shape[0])
            )
            print("final_store length = {0}".format(df_stored.shape[0]))
            print("=" * 50)

    # Export subsampled dataframe as CSV
    # Two added columns:
    # 1. issue_year (dtype=int)
    # 2. issue_month (dtype=int)
    if csv_type:
        df_stored.set_index("id").to_csv(path_or_buf=output_path, sep=",")
    else:
        df_stored.to_csv(path_or_buf=output_path, sep=",")

    return print("SUBSAMPLING COMPLETE")

In [52]:
def month_subsample_idx(df, frac, mo, n_floor):
    
    """
    Purpose: 
        Generate idx of subsampled observations for a given month
    Input:
        df = df to be sub-sampled
        frac = fraction of observations to be sub-sampled each year
        mo = integer of month
        n_floor = set min numb. of subsample observations [n=20, per month]
    Return:
        subsample_idx = indices corresponding to randomly generated subsample (for given month)
    """
    
    mo_idx = np.where(df.issue_month == mo)[0].tolist() #idx of observations occuring in month
    n_orig = len(mo_idx) #total n of observations
    n_sub = math.ceil(n_orig*frac) #n of subsampled observations
    
    #Set n_sub = 20 (per month) if n_floor selected by user
    if (n_floor & (n_orig>=20) & (n_sub<20)):
        new_sub = 20
    elif(n_floor & (n_orig<20) & (n_sub<20)):
        new_sub = n_orig
    else:
        new_sub = n_sub   
        
    subsample_idx = random.sample(mo_idx, new_sub) #randomly generated subsampled indices
    
    return subsample_idx


### Subsample accepted.csv

- chucksize=200k is arbitrary

In [41]:
%%time
subsampling(
    input_path="../data/accepted_2007_to_2018Q4.csv", #input file
    output_path="../data/accepted_subsampled_5percent.csv", #output file
    csv_type=True, #accepted.csv
    frac=0.05, #github limited to 100mb files
    chunksize=200000, #arbitrary
    n_floor=True, #set min numb. subsamples per month [n=20, per month]
    printDetails=True
)

year = 2007
year_store length = 140
final_store length = 140
year = 2008
year_store length = 241
final_store length = 381
year = 2009
year_store length = 289
final_store length = 670
year = 2010
year_store length = 634
final_store length = 1304
year = 2011
year_store length = 1091
final_store length = 2395
year = 2012
year_store length = 2674
final_store length = 5069
year = 2013
year_store length = 6745
final_store length = 11814
year = 2014
year_store length = 11787
final_store length = 23601
year = 2015
year_store length = 21062
final_store length = 44663
year = 2016
year_store length = 21726
final_store length = 66389
year = 2017
year_store length = 22186
final_store length = 88575
year = 2018
year_store length = 24768
final_store length = 113343
SUBSAMPLING COMPLETE
CPU times: user 14min 27s, sys: 2min 10s, total: 16min 37s
Wall time: 17min 16s


<IPython.core.display.Javascript object>

In [42]:
# Test to see if works
pd.read_csv("../data/accepted_subsampled_5percent.csv", sep=",", nrows=10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,issue_year,issue_month
0,88046,NaN,4400.0,4400.0,1400.0,36 months,9.64,141.25,B,B4,Brick Township board of education,2 years,MORTGAGE,30000.0,Not Verified,Jun-2007,Does not meet the credit policy. Status:Fully ...,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,VISA,087xx,NJ,3.72,0.0,Jul-2004,695.0,699.0,0.0,0.0,0.0,4.0,0.0,3493.0,63.5,5.0,f,0.0,0.0,5084.724868,1617.87,4400.00,684.72,0.00,0.0,0.00,Jun-2010,143.28,Jul-2010,May-2018,549.0,545.0,NaN,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2007,6
1,74505,NaN,2000.0,2000.0,225.0,36 months,9.96,64.50,B,B5,NaN,< 1 year,RENT,6000.0,Not Verified,Jun-2007,Does not meet the credit policy. Status:Fully ...,n,https://lendingclub.com/browse/loanDetail.acti...,I just need enough money to make it until the ...,other,Summer stuff,325xx,FL,10.00,NaN,NaN,685.0,689.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,0.0,0.0,2322.408993,261.27,2000.00,322.41,0.00,0.0,0.00,Jul-2010,0.84,Aug-2010,Jul-2010,594.0,590.0,NaN,NaN,1.0,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

<IPython.core.display.Javascript object>

### Subsample declined.csv

- chucksize=200k is arbitrary

In [55]:
%%time
subsampling(
    input_path="../data/rejected_2007_to_2018Q4.csv", #input file
    output_path="../data/rejected_subsampled_5percent.csv", #output file
    csv_type=False, #rejected.csv
    frac=0.05, #github limited to 100mb files
    chunksize=200000, #arbitrary
    n_floor=True, #set min numb. subsamples per month [n=20, per month]
    printDetails=True
)

year = 2007
year_store length = 305
final_store length = 305
year = 2008
year_store length = 1285
final_store length = 1590
year = 2009
year_store length = 2856
final_store length = 4446
year = 2010
year_store length = 5633
final_store length = 10079
year = 2011
year_store length = 10895
final_store length = 20974
year = 2012
year_store length = 16869
final_store length = 37843
year = 2013
year_store length = 38053
final_store length = 75896
year = 2014
year_store length = 96692
final_store length = 172588
year = 2015
year_store length = 142974
final_store length = 315562
year = 2016
year_store length = 238498
final_store length = 554060
year = 2017
year_store length = 353636
final_store length = 907696
year = 2018
year_store length = 474845
final_store length = 1382541
SUBSAMPLING COMPLETE
CPU times: user 17min 2s, sys: 3min 5s, total: 20min 8s
Wall time: 20min 9s


In [54]:
# Test to see if works
pd.read_csv("../data/rejected_subsampled_5percent.csv", sep=",", nrows=10)

,Unnamed: 0,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code,issue_year,issue_month
0,18,15000.0,2007-05-29,Cancer is Killing My Credit,680.0,10%,432xx,OH,< 1 year,0.0,2007,5
1,8,3000.0,2007-05-28,title?,694.0,10%,808xx,CO,4 years,0.0,2007,5
2,42,7500.0,2007-05-31,maryk2001,524.0,61.47%,316xx,GA,5 years,0.0,2007,5
3,37,15000.0,2007-05-30,2dozen,474.0,1.55%,453xx,OH,4 years,0.0,2007,5
4,39,2500.0,2007-05-31,mbcoon,593.0,18.43%,790xx,TX,1 year,0.0,2007,5
5,41,3000.0,2007-05-31,danthe83,536.0,0%,551xx,MN,< 1 year,0.0,2007,5
6,10,3900.0,2007-05-28,need to consolidate,710.0,10%,705xx,LA,10+ years,0.0,2007,5
7,46,1000.0,2007-05-31,jprice1542,0.0,4.61%,373xx,TN,< 1 year,0.0,2007,5
8,32,7000.0,2007-05-30,big fat lender,638.0,17.73%,941xx,CA,< 1 year,0.0,2007,5
9,12,3000.0,2007-05-28,bmoore5110,688.0,10%,190xx,PA,< 1 year,0.0,2007,5
